In [1]:
"""
Quora pairs Kaggle competition
Feature Engineering
@author: Luis Duque
"""
import os
import time
import pandas as pd
import numpy as np
import csv
import string
import re, math
from string import punctuation
from difflib import SequenceMatcher
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from operator import xor
from IPython.display import clear_output

In [2]:
## Word2Vect and NLTK models loading
import gensim
import nltk
from nltk.tag import pos_tag
from gensim.models import Word2Vec
nltk.download('averaged_perceptron_tagger')
model = gensim.models.KeyedVectors.load_word2vec_format('./data/GoogleNews-vectors-negative300.bin', binary=True)
model.init_sims(replace=True)  # Normalizes the vectors in the word2vec class.

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/gpu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
def LoadDictionary(): # Loads Dictionary.csv 
    Dictionary ={}
    DictionaryDf = pd.DataFrame.from_csv("Dictionary.csv")
    Words = DictionaryDf['Words']
    Quotient1 = DictionaryDf[ 'Quotient1']
    Quotient2 = DictionaryDf[ 'Quotient2']
    for i in range(0, len(Words)):   
        Dictionary[Words[i]] = [Quotient1[i], Quotient2[i]]
    return Dictionary    
Dictionary = LoadDictionary();                ## Loading Dictionary.csv 

In [7]:
#########################################################################################################################
###################### FEATURE ENGINEERING ##############################################################################
#########################################################################################################################
WORD = re.compile(r'\w+') ## used when computing the cosine distance
StopWords = {"ourselves", "hers", "between", "yourself", "but", "again", "there", "about", "once", "during", "out", "very", "having", "with", "they", "own", "an", "be", "some", "for", "do", "its", "yours", "such", "into", "of", "most", "itself", "other", "off", "is", "s", "am", "or", "who", "as", "from", "him", "each", "the", "themselves", "until", "below", "are", "we", "these", "your", "his", "through", "don", "nor", "me", "were", "her", "more", "himself", "this", "down", "should", "our", "their", "while", "above", "both", "up", "to", "ours", "had", "she", "all", "no", "when", "at", "any", "before", "them", "same", "and", "been", "have", "in", "will", "on", "does", "yourselves", "then", "that", "because", "what", "over", "why", "so", "can", "did", "not", "now", "under", "he", "you", "herself", "has", "just", "where", "too", "only", "myself", "which", "those", "i", "after", "few", "whom", "t", "being", "if", "theirs", "my", "against", "a", "by", "doing", "it", "how", "further", "was", "here", "than"} 
PiecesOfSpeech = ["CC", "CD", "DT", "EX", "FW", "IN", "JJ", "JJR", "JJS", "LS", "MD", "NN", "NNP", "NNPS", "NNS", "PDT", "POS", "PRP", "PRP$", "RB", "RBR", "RBS", "RP", "SYM", "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "WDT", "WP", "WP$", "WRB"]
PartsOfSpeech = PiecesOfSpeech + ["ADJETIVES", "NOUNS", "PROPERNOUNS", "PRONOUNS", "ADVERBS", "VERBS", "WH"]
RelevantPartsOfSpeech =  ["ADJETIVES", "NOUNS", "PROPERNOUNS", "PRONOUNS", "ADVERBS", "VERBS", "WH"]
#StartWords = ["what", "how", "why", "is", "which", "can", "i", "who", "do", "where", "if", "does", "are", "should", "when", "will", "did", "would", "has", "have", "was", "could", "for", "after", "am", "at", "were", "from" , "with"]
StartWords = ["what", "how", "why", "is", "which", "can", "i", "who", "do", "where"]

#####
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def FuzzySame(Word1, Word2):   ## THIS SHOULD BE IMPROVED
    ## we compare them in a more straightforward way
    comparison = similar(Word1, Word2)
    #print "comparing ", Word1, " and ", Word2, " = \t", comparison, "(usign similar)"

    if comparison > 0.8:
        return 1
    else: 
        return 0
    
def FuzzyWordInList(Word, List):
    if Word in List: 
        return 1
    
    for cWord in List:
        if FuzzySame(Word, cWord):
            return 1;
    return 0


## This function is not really fuzzy but if it was the generation of the dictionary would be too slow.
def FuzzyFindRepresentative(Word, List): ## will also work for if List is a dictionary, 
    if Word in List: 
        return Word
    else:
        return ""

    
def FuzzyIntersection(Bag1, Bag2):
    Intersection = []
    for Word in Bag1:
        if FuzzyWordInList(Word, Bag2):
            Intersection.append(Word)
    return Intersection


def FuzzyUnion(Bag1, Bag2):
    Set1 = set(Bag1)-set(FuzzyIntersection(Bag1, Bag2))
    Set2 = set(Bag2)
    return list(Set1.union(Set2))


def FuzzyWordsInOneList(Bag1, Bag2):
    Lista = []
    for Word in Bag1:
        if not FuzzyWordInList(Word, Bag2):
            Lista.append(Word)
        
    for Word in Bag2:
        if not FuzzyWordInList(Word, Bag1):
            Lista.append(Word)
    return Lista


#########################################################################################################################
### Very basic features
def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation+"’")

def strip_punctuation_nums(s):
    s = s.lower()
    return ''.join(c for c in s if c not in punctuation+"’0123456789")

## Auxiliar function that converts a string of characters into a Bag of words
def StringToList_Nonums(Doc):
    nopunctuationlower = strip_punctuation_nums(Doc)
    tokens = nopunctuationlower.split()
    return tokens

def num_words(document):
    tokens = set(document.split())
    return len(tokens)

def num_stopwords(document):
    filtered_tokens = []
    nopunctuation = strip_punctuation(document)
    tokens = set(nopunctuation.lower().split())
    filtered_sentence = [w for w in tokens if not w in StopWords]
    return len(filtered_sentence)

def num_capitals(document):
    capitals = "".join([c for c in document if c.isupper()])
    return len(capitals)

def num_questionmarks(document):
    questions = "".join([c for c in document if c=="?"])
    return len(questions)

def num_exclamationmarks(document):
    exclamations = "".join([c for c in document if c=="!"])
    return len(exclamations)

def num_quotations(document):
    quotations = "".join([c for c in document if c=='"'  or c =="'" or c =="‘" or c =="’" or c =="“" or c =="”" ])
    return len(quotations)

def num_dots(document):
    dots = "".join([c for c in document if c=='.' ])
    return len(dots)

def num_commas(document):
    commas = "".join([c for c in document if c==',' ])
    return len(commas)

def first_word(document):
    nopunctuation = strip_punctuation(document)
    tokens = list(nopunctuation.lower().split())
    if len(tokens)== 0:
        return ""
    return tokens[0]


def first_word_features(Doc):
    word = first_word(Doc)
    F = [0]*len(StartWords)
    
    if word in StartWords:
        location = StartWords.index(word)
        F[location]= 1
    return F


def GetPartsOfSpeech(Doc):
    Parts = {} 
    for s in PartsOfSpeech:
        Parts[s] = []
    
    tagged1 = pos_tag(Doc.split())
    
    for word,pos in tagged1:
        if pos in PiecesOfSpeech:
            Parts[pos].append(  strip_punctuation( word.lower()) )
    Parts["ADJETIVES"] =  Parts["JJ"] + Parts["JJR"] + Parts["JJS"]
    Parts["NOUNS"] =  Parts["NN"] + Parts["NNS"]
    Parts["PROPERNOUNS"] = Parts["NNP"] + Parts["NNPS"] 
    Parts["PRONOUNS"] = Parts["PRP"] + Parts["PRP$"]
    Parts["ADVERBS"] = Parts["RB"] + Parts["RBR"] + Parts["RBS"]
    Parts["VERBS"] = Parts["VB"] + Parts["VBD"] + Parts["VBG"] + Parts["VBN"] + Parts["VBP"] + Parts["VBZ"]
    Parts["WH"] =  Parts["WDT"] + Parts["WP"] + Parts["WP$"] + Parts["WRB"]       
    return Parts

#########################################################################################################################
### Definition of the distances relevant to us: JaccardList1List2, CosineList1List2, WordMoversList1List2
def JaccardList1List2(List1, List2):   
    List1.append("a")  # avoids divisions by zero
    List2.append("a")
    Set1 = set(List1)
    Set2 = set(List2)
    intersection = Set1.intersection(Set2)
    union = Set1.union(Set2)
    return float(len(intersection))/(len(union))    

def WordMoversList1List2(List1, List2):
    List1.append("the")
    List2.append("the")
    Set1 = set(List1)
    Set2 = set(List2)
    Out = model.wmdistance(Set1, Set2)
    if Out == float("Inf"):
        print "WARNING!"
    return Out

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    text = text + " a"
    words = WORD.findall(text)
    return Counter(words)

def CosineList1List2(List1, List2):
    text1 = ""
    text2 = ""
    for w in List1:
        text1 = text1 + w + " "
    for w in List2:
        text2 = text2 + w+ " " 
    return get_cosine(text_to_vector(text1), text_to_vector(text2))

#########################################################################################################################
### Functions that create many features provided that we have a defined distance between words
def DistanceStr1Str2(Str1, Str2, DistanceSet1Set2):
    nopunctuation1 = strip_punctuation(Str1)
    nopunctuation2 = strip_punctuation(Str2)
    tokens1 = set(nopunctuation1.lower().split())
    tokens2 = set(nopunctuation2.lower().split())
    return DistanceSet1Set2(tokens1, tokens2)
    
def DistanceFeatures(Doc1, Doc2, DistanceList1List2):
    PartsofDoc1 = GetPartsOfSpeech(Doc1)
    PartsofDoc2 = GetPartsOfSpeech(Doc2)
    Bag1 = strip_punctuation( Doc1.lower()).split()
    Bag2 = strip_punctuation( Doc2.lower()).split()
    Bag1NonStop = [w for w in Bag1 if w not in StopWords]
    Bag2NonStop = [w for w in Bag2 if w not in StopWords]
    
    #We append the Distance  between the two documents and the distance without include stopwords
    Distance = [DistanceList1List2(Bag1, Bag2), DistanceList1List2(Bag1NonStop, Bag2NonStop)] 
    
    #We append the Distance between verbs, adverbs, nouns, ... (Only the most relevant ones: verbs, adverbs, ...)
    for part in RelevantPartsOfSpeech:
        Distance.append( DistanceList1List2( PartsofDoc1[part], PartsofDoc2[part])  )
    return Distance

def JaccardFeatures(Doc1, Doc2):
    return DistanceFeatures(Doc1, Doc2, JaccardList1List2)

def WordMoversFeatures(Doc1, Doc2):
    return DistanceFeatures(Doc1, Doc2, WordMoversList1List2)

def CosineFeatures(Doc1, Doc2):
    return DistanceFeatures(Doc1, Doc2, CosineList1List2)


#########################################################################################################################
#### Features involving the words that are in exactly one questions or in both
def InOneBagFeatures(Doc1, Doc2, lenmaxintersection, Dictionary):  
    Bag1 = StringToList_Nonums(Doc1)
    Bag2 = StringToList_Nonums(Doc2)
    InOneBag = FuzzyWordsInOneList(Bag1, Bag2)        ## words in both bags
    
    OneBagQuotients = []
    for cWord in InOneBag:
        if cWord in Dictionary:
            cQuotient = Dictionary[cWord]
            OneBagQuotients.append(cQuotient[0])          ## this is Quotient1.  
        
    OneBagQuotients.sort(reverse=True)           
    len1 = len(OneBagQuotients)
    
    while len(OneBagQuotients)< lenmaxintersection:   ## We truncate OneBagQuotients to have exactly lenmaxintersection elements 
        OneBagQuotients.append(0)

    while len(OneBagQuotients)> lenmaxintersection:
        del OneBagQuotients[-1]
    
    OneBagQuotients.append(len1)                      ## We append the size of the intersection
    
    return OneBagQuotients

def InBothBagsFeatures(Doc1, Doc2, maxelements, Dictionary):       
    Bag1 = StringToList_Nonums(Doc1)
    Bag2 = StringToList_Nonums(Doc2)
    InBothBags = FuzzyIntersection(Bag1, Bag2)         ## words in both bags
    
    BothBagsQuotients =[]

    for cWord in InBothBags:
        if cWord in Dictionary:
            cQuotient = Dictionary[cWord]
            BothBagsQuotients.append(cQuotient[1])  ## this is Quotient2.

    BothBagsQuotients.sort(reverse=True)   ## Sort the coefficients.

    len2 = len(BothBagsQuotients)           

    while len(BothBagsQuotients)< maxelements:
        BothBagsQuotients.append(0)

    while len(BothBagsQuotients)> maxelements:
        del BothBagsQuotients[-1]
        
    BothBagsQuotients.append(len2)
    return BothBagsQuotients


#########################################################################################################################
### Generation of all the features of a pair of questions (Doc1, Doc2)
def AllFeatures(Doc1, Doc2):
    Feats = [len(Doc1), len(Doc2)]                                             #Length of question (2)  
    #Feats = Feats + [num_words(Doc1), num_words(Doc2)]                        #Number of words (2)
    #Feats = Feats + [num_words(Doc1)-num_stopwords(Doc1), num_words(Doc2)-num_stopwords(Doc2)] #Number of words except for stopwords (2) 
    Feats = Feats + first_word_features(Doc1) + first_word_features(Doc2)     #Indicators for questions starting with “Are”, “Can”, “How” etc… (10)  
    #Feats = Feats + [num_exclamationmarks(Doc1), num_exclamationmarks(Doc2)]  #Number of exclamation marks (2)
    Feats = Feats + [num_quotations(Doc1), num_quotations(Doc2)]              #Number of quotes (2) 
    Feats = Feats + [num_questionmarks(Doc1), num_questionmarks(Doc2)]        #Number of question marks (2) 
    Feats = Feats + [num_capitals(Doc1), num_capitals(Doc2)]                  #Number of capital letters (2) 
    #Feats = Feats + [num_dots(Doc1), num_dots(Doc2)]                          #Number of dots (2)  
    #Feats = Feats + [num_commas(Doc1), num_commas(Doc2)]                      #Number of commas (2)  
    #Feats = Feats + JaccardFeatures(Doc1, Doc2)                               #Jaccard Features (many)
    Feats = Feats + WordMoversFeatures(Doc1, Doc2)                            #Word Mover’s Features(many)
    #Feats = Feats + CosineFeatures(Doc1, Doc2)                                #Cosine Features(many)
    Feats = Feats + InBothBagsFeatures(Doc1, Doc2, 5, Dictionary)              #Features involving words that are in both bags
    Feats = Feats + InOneBagFeatures(Doc1, Doc2, 5, Dictionary)                #Features involving words that are only in one of the bags
    return Feats

#########################################################################################################################
### Creates the data frame with the Features we will use in the classification##################################
def FeatureEngineering(df, filename):
    Features = []     
    for index, row in df.iterrows():
        Doc1 = str(row['question1'])   
        Doc2 = str(row['question2']) 
        Features.append(AllFeatures(Doc1, Doc2))
        if (index%10000) ==0:
            print "Current Row=", index
    labels = []
    for i in range(0, len(Features[0])):
        labels.append("F"+str(i))
    dfFeatures = pd.DataFrame.from_records(Features, columns=labels) 
    dfFeatures.to_csv(filename, header = True) 
    return dfFeatures

In [5]:
#################################################################################################################
###############################             MAIN                 ################################################
#################################################################################################################

In [8]:
dftrain = pd.DataFrame.from_csv("./data/train.csv")  ## Loading data 
trainfeaturesdf = FeatureEngineering(dftrain, "Mtrain.csv") ## Feature engineering

Current Row= 0
Current Row= 10000
Current Row= 20000
Current Row= 30000
Current Row= 40000
Current Row= 50000
Current Row= 60000
Current Row= 70000
Current Row= 80000
Current Row= 90000
Current Row= 100000
Current Row= 110000
Current Row= 120000
Current Row= 130000
Current Row= 140000
Current Row= 150000
Current Row= 160000
Current Row= 170000
Current Row= 180000
Current Row= 190000
Current Row= 200000
Current Row= 210000
Current Row= 220000
Current Row= 230000
Current Row= 240000
Current Row= 250000
Current Row= 260000
Current Row= 270000
Current Row= 280000
Current Row= 290000
Current Row= 300000
Current Row= 310000
Current Row= 320000
Current Row= 330000
Current Row= 340000
Current Row= 350000
Current Row= 360000
Current Row= 370000
Current Row= 380000
Current Row= 390000
Current Row= 400000


In [ ]:
print trainfeaturesdf.head()
dftrain = {}
trainfeaturesdf = {}

    qid1  qid2                                          question1  \
id                                                                  
0      1     2  What is the step by step guide to invest in sh...   
1      3     4  What is the story of Kohinoor (Koh-i-Noor) Dia...   
2      5     6  How can I increase the speed of my internet co...   
3      7     8  Why am I mentally very lonely? How can I solve...   
4      9    10  Which one dissolve in water quikly sugar, salt...   

                                            question2  is_duplicate  
id                                                                   
0   What is the step by step guide to invest in sh...             0  
1   What would happen if the Indian government sto...             0  
2   How can Internet speed be increased by hacking...             0  
3   Find the remainder when [math]23^{24}[/math] i...             0  
4             Which fish would survive in salt water?             0  


In [ ]:
# Feature creation
dftest = pd.DataFrame.from_csv("./data/test.csv")
testfeaturesdf = FeatureEngineering(dftest, "Mtest.csv")
dftest ={}
testfeaturesdf={}

Current Row= 0
Current Row= 10000
Current Row= 20000
Current Row= 30000
Current Row= 40000
Current Row= 50000
Current Row= 60000
Current Row= 70000
Current Row= 80000
Current Row= 90000
Current Row= 100000
Current Row= 110000
Current Row= 120000
Current Row= 130000
Current Row= 140000
Current Row= 150000
Current Row= 160000
Current Row= 170000
Current Row= 180000
Current Row= 190000
Current Row= 200000
Current Row= 210000
Current Row= 220000
Current Row= 230000
Current Row= 240000
Current Row= 250000
Current Row= 260000
Current Row= 270000
Current Row= 280000
Current Row= 290000
Current Row= 300000
Current Row= 310000
Current Row= 320000
Current Row= 330000
Current Row= 340000
Current Row= 350000
Current Row= 360000
Current Row= 370000
Current Row= 380000
Current Row= 390000
Current Row= 400000
Current Row= 410000
Current Row= 420000
Current Row= 430000
Current Row= 440000
Current Row= 450000
Current Row= 460000
Current Row= 470000
Current Row= 480000
Current Row= 490000
Current Row= 5